In [12]:
pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=701a135eb72a98e4522300a02ff6f56891ce3f8dd6efc08272bc44b1ad8aa35c
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [13]:
import pandas as pd
from pandasql import sqldf
import numpy as np

In [14]:
df = pd.read_csv('/content/RC/challenge_data.csv')

In [15]:
#Crear funcion para correr queries
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
#Crear funcion para extraer medias y medianas mediante un query.
#Las medianas no se pueden calcular directamente en esta version de SQL
#por lo que se calcula el valor del percentil 50 de cada variable,
#en este caso la formula funciona mientras la cardinalidad del campo
#en cuestion sea impar ya que los tres campos para los que se utiliza en
#este ejercicio cumplen con esta condicion

def mean_and_median(var, df):
  #Generar dataframe con cardinalidad de cada categoria sin considerar los nulls
  #para encontrar la mediana
  df_impago_counts_mora = df[df[var].notnull()]['impago'].value_counts().reset_index(name = 'counts')

  #Calcular el punto medio de cada grupo
  fifty_percentile_impago_0 = np.ceil(df_impago_counts_mora.iloc[0,1]/2)
  fifty_percentile_impago_1 = np.ceil(df_impago_counts_mora.iloc[1,1]/2)

    # Query
  query = f"""
  with means as (SELECT
    impago,
    avg({var}) as avg
  FROM
    df
  group by 1
  LIMIT 10),

  median_0 as(
      SELECT
        "0" as impago,
        {var} as median
      FROM
        df
      WHERE impago ="0"
      AND {var} not null
      ORDER BY 2 desc
      LIMIT 1
      OFFSET {fifty_percentile_impago_0}
  ),

  median_1 as(
      SELECT
        "1" as impago,
        {var} as median
      FROM
        df
      WHERE impago ="1"
      AND {var} not null
      ORDER BY 2 desc
      LIMIT 1
      OFFSET {fifty_percentile_impago_1}
  ),

  medians as (
    SELECT * FROM median_0
    UNION ALL
    SELECT * FROM median_1
  )

  SELECT
  means.impago,
  avg,
  median
  FROM
  medians
  JOIN
  means
  on medians.impago = means.impago
  """

  # Ejecutar query y mostrar resultado
  result_df = pysqldf(query)
  return(result_df)

Query 1

In [23]:
mean_and_median('pago_para_no_generar_interes', df)

,index,impago,avg,median
0,0,0,4706.034285,3174.19
1,1,1,9366.711322,7149.63


Query 2

In [15]:
# Query 2

#Dado que no existen ids duplicados, unicamente se considera
#la id con mayor cuentas activas mediante ordenarlas de acuerdo
#al campo num_cuentas_activas y extrayendo el valor mas alto.
#Se reviso que no existan empates en el top 1.
query_2 = """
SELECT
  id,
  num_cuentas_activas
FROM
  df
order by 1 desc
LIMIT 1
"""

# Execute the query
result_df = pysqldf(query_2)
print(result_df)

           id  num_cuentas_activas
0  zzz8o18JNC                   11


El cliente con mayor número de cuentas activas cuenta con 11 cuentas.

Query 3

In [24]:
mean_and_median('total_credit', df)

,index,impago,avg,median
0,0,0,150878.292345,35300.0
1,1,1,71732.619328,20000.0


Query 4

In [26]:
mean_and_median('monto_en_mora',df)

,index,impago,avg,median
0,0,0,3001.287089,0.0
1,1,1,15936.012981,6743.0


Query 5

In [56]:
#Ya que pandasql no posee un cálculo para varianzas, covarianzas, ni regresiones
#lineales, se produce un calculo de covarianza que tambien se comparará con
#los resultados de un modelo lineal no producido por SQL

#cálculo de medias
def mean_and_variance(var):
    query_mean = f"""
    SELECT
      avg({var}) as mean
    FROM
      df
    WHERE
      {var} is not null
    """
    mean = pysqldf(query_mean)

    #return mean

#def variance(var):

    #mean_var = mean(var)
    #mean_var = mean_var.iloc[0,0]
    mean = mean.iloc[0,0]

    query_variance = f"""
    SELECT
      SUM(({var} - {mean}) * ({var} - {mean})) / (COUNT({var}) - 1) as variance
    FROM
      df
    WHERE
      {var} is not null
    """

    variance = pysqldf(query_variance)

    variance = variance.iloc[0,0]
    return (mean, variance)

def correlation(var_1, var_2):

    stats_1 = mean_and_variance(var_1)
    mean_1 = stats_1[0]
    variance_1 = stats_1[1]
    stats_2 = mean_and_variance(var_2)
    mean_2 = stats_2[0]
    variance_2 = stats_2[1]

    query_covariance = f"""
      SELECT
        SUM(({var_1} - {mean_1}) * ({var_2} - {mean_2})) / (COUNT({var_1}) - 1) as covariance
      FROM
        df
      WHERE
        {var_1} is not null
      AND
        {var_2} is not null
      """

    cov = pysqldf(query_covariance)

    cov = cov.iloc[0,0]

    corr = cov/((variance_1**0.5) * (variance_2**0.5))

    return corr

Calculando la correlación basada en los cálculos de medias y varianzas, se obtiene un coeficiente muy bajo de 0.048. Esto sugiere una baja relación entre el número de cuentas activas de los clientes y su monto en mora. Sin embargo, esto no es un análisis estadístico robusto y para investigarse se necesitarían paquetes adicionales a pandasql.

In [57]:
correlation('monto_en_mora', 'num_cuentas_activas')

0.048690582175933445